In [52]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.linear_model import LinearRegression

import numpy as np

In [53]:
fuel = pd.read_csv('fuel.csv')
fuel.drop(columns={'Unnamed: 0'}, inplace=True)

In [54]:
fuel.isna().sum()

EngDispl               0
NumCyl                 0
Transmission           0
FE                     0
AirAspirationMethod    0
NumGears               0
TransLockup            0
TransCreeperGear       0
DriveDesc              0
IntakeValvePerCyl      0
ExhaustValvesPerCyl    0
CarlineClassDesc       0
VarValveTiming         0
VarValveLift           0
dtype: int64

In [55]:
y = fuel['FE']
X = fuel[['EngDispl', 'NumCyl', 'Transmission', 'AirAspirationMethod','NumGears', 'TransLockup', 
          'TransCreeperGear', 'DriveDesc','IntakeValvePerCyl', 'ExhaustValvesPerCyl', 'CarlineClassDesc',
          'VarValveTiming', 'VarValveLift']]

In [56]:
preprocessing = make_column_transformer(
                (StandardScaler(),make_column_selector(dtype_include=np.number)),
                (OneHotEncoder(sparse=True), make_column_selector(dtype_include=object)),
)

In [57]:
X = preprocessing.fit_transform(X)
y = np.log(y)

X_train, X_valid , y_train, y_valid = train_test_split(X,y,test_size=0.2,random_state=420)

In [62]:
input_shape = [X_train.shape[1]]

In [66]:
model = keras.Sequential([
        # First layers
        layers.Dense(units=32, activation='relu', input_shape=input_shape),
        # Second Layers
        layers.Dense(units=32,activation='relu'),
        # Final layers
        layers.Dense(units=1),
]
)

In [71]:
model.compile(optimizer='adam',loss='mae',)

In [74]:
history = model.fit(
            X_train, y_train,
            validation_data=(X_valid,y_valid),
            epochs=10,
            batch_size=50,
)

Epoch 1/10


InvalidArgumentError:  TypeError: 'SparseTensor' object is not subscriptable
Traceback (most recent call last):

  File "/Users/bt/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 242, in __call__
    return func(device, token, args)

  File "/Users/bt/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 131, in __call__
    ret = self._func(*args)

  File "/Users/bt/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/autograph/impl/api.py", line 302, in wrapper
    return func(*args, **kwargs)

  File "/Users/bt/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/data_adapter.py", line 513, in py_method
    return [slice_array(inp) for inp in flat_inputs]

  File "/Users/bt/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/data_adapter.py", line 513, in <listcomp>
    return [slice_array(inp) for inp in flat_inputs]

  File "/Users/bt/opt/anaconda3/lib/python3.8/site